# Final Project

## Available Assets for Portfolio Optimization
Here is a list of the available assets to consider in the portfolio:

<table>
  <thead>
    <tr>
      <th>Fund Name</th>
      <th>Ticker</th>
      <th>Inception Date</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>JPMorgan Equity Income Fund - Class R6</td>
      <td><a href="https://finance.yahoo.com/quote/OIEJX/history/" target="_blank">OIEJX</a></td>
      <td>Jan 31, 2012</td>
    </tr>
    <tr>
      <td>State Street Equity 500 Index K</td>
      <td><a href="https://finance.yahoo.com/quote/SSSYX/history/" target="_blank">SSSYX</a></td>
      <td>Sep 18, 2014</td>
    </tr>
    <tr>
      <td>T. Rowe Price Dividend Growth</td>
      <td><a href="https://finance.yahoo.com/quote/PRDGX/history/" target="_blank">PRDGX</a></td>
      <td>Dec 31, 1992</td>
    </tr>
    <tr>
      <td>American Funds Growth Fund of Amer R6</td>
      <td><a href="https://finance.yahoo.com/quote/RGAGX/history/" target="_blank">RGAGX</a></td>
      <td>May 1, 2009</td>
    </tr>
    <tr>
      <td>Vanguard Mid Cap Index Admiral</td>
      <td><a href="https://finance.yahoo.com/quote/VIMAX/history/" target="_blank">VIMAX</a></td>
      <td>Nov 12, 2001</td>
    </tr>
    <tr>
      <td>Vanguard Small Cap Value Index Admiral</td>
      <td><a href="https://finance.yahoo.com/quote/VSIAX/history/" target="_blank">VSIAX</a></td>
      <td>Sep 27, 2011</td>
    </tr>
  </tbody>
</table>

Historical data for these assets can be found on [Yahoo Finance website](https://finance.yahoo.com)

Since the portfolio optimization problem requires grouping the time series of each asset into matrix $R$, they must be considered during the same periods. For this reason, only returns starting from the most restrictive case are considered, i.e., **from Sep 18, 2014 until Aug 27, 2025**.

Data sources:
<table>
  <thead>
    <tr>
      <th>Ticker</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td><a href="https://finance.yahoo.com/quote/OIEJX/history/?period1=1410998400&period2=1756339200" target="_blank">OIEJX</a></td>
    </tr>
    <tr>
      <td><a href="https://finance.yahoo.com/quote/SSSYX/history/?period1=1410998400&period2=1756339200" target="_blank">SSSYX</a></td>
    </tr>
    <tr>
      <td><a href="https://finance.yahoo.com/quote/PRDGX/history/?period1=1410998400&period2=1756339200" target="_blank">PRDGX</a></td>
    </tr>
    <tr>
      <td><a href="https://finance.yahoo.com/quote/RGAGX/history/?period1=1410998400&period2=1756339200" target="_blank">RGAGX</a></td>
    </tr>
    <tr>
      <td><a href="https://finance.yahoo.com/quote/VIMAX/history/?period1=1410998400&period2=1756339200" target="_blank">VIMAX</a></td>
    </tr>
    <tr>
      <td><a href="https://finance.yahoo.com/quote/VSIAX/history/?period1=1410998400&period2=1756339200" target="_blank">VSIAX</a></td>
    </tr>
  </tbody>
</table>
The service is behind a paywall since early 2025, but data is still accessible inspecting the html elements of the page.

In [23]:
# Import necessary libraries
import os
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import cvxpy as cp

## Ingestion

In [24]:
def load_html_to_df(html_file):
    """
    Reads a Yahoo Finance HTML file and returns a cleaned pandas DataFrame.
    The columns (except 'date') are automatically prefixed with the file name.
    """
    with open(html_file, 'r') as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, 'lxml')
    table_rows = soup.find_all('tr')

    data = []
    for row in table_rows:
        row_data = [cell.text.strip() for cell in row.find_all(['td', 'th'])]
        if row_data:
            data.append(row_data)

    columns = ['date', 'open', 'high', 'low', 'close', 'adj_close', 'volume']
    df = pd.DataFrame(data, columns=columns)
    df['date'] = pd.to_datetime(df['date'])

    prefix = os.path.splitext(os.path.basename(html_file))[0]
    # Rename columns except 'date' by prefixing with the file name
    df.columns = ['date'] + [f"{prefix}_{col}" for col in columns[1:]]

    for col in df.columns[1:]:
        df[col] = pd.to_numeric(df[col], errors='coerce', downcast='float')
    df = df.sort_values('date', ascending=True).reset_index(drop=True)
    
    return df

In [25]:
oiejx = load_html_to_df('../data/oiejx.html')
sssyx = load_html_to_df('../data/sssyx.html')
prdgx = load_html_to_df('../data/prdgx.html')
rgagx = load_html_to_df('../data/rgagx.html')
vimax = load_html_to_df('../data/vimax.html')
vsiax = load_html_to_df('../data/vsiax.html')

## Exploratory Data Analysis

In [26]:
sssyx

,date,sssyx_open,sssyx_high,sssyx_low,sssyx_close,sssyx_adj_close,sssyx_volume
0,2014-09-18,170.899994,170.899994,170.899994,170.899994,126.000000,NaN
1,2014-09-19,170.800003,170.800003,170.800003,170.800003,125.930000,NaN
2,2014-09-22,169.399994,169.399994,169.399994,169.399994,124.900002,NaN
3,2014-09-23,168.500000,168.500000,168.500000,168.500000,124.230003,NaN
4,2014-09-24,169.800003,169.800003,169.800003,169.800003,125.190002,NaN
...,...,...,...,...,...,...,...
2766,2025-08-21,479.130005,479.130005,479.130005,479.130005,479.130005,NaN
2767,2025-08-22,486.440002,486.440002,486.440002,486.440002,486.440002,NaN
2768,2025-08-25,484.369995,484.369995,484.369995,484.369995,484.369995,NaN
2769,2025-08-26,486.399994,486.399994,486.399994,486.399994,486.399994,NaN


We can see that all columns are actually the same value. This is because mutual funds are priced once per day at their net asset value (NAV), not traded during the day like stocks. Since there’s only one price, data providers copy the NAV into all fields (open, high, low, close, adj close), and volume is usually zero or blank.

In [27]:
sssyx.info()
sssyx.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2771 entries, 0 to 2770
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             2771 non-null   datetime64[ns]
 1   sssyx_open       2752 non-null   float32       
 2   sssyx_high       2752 non-null   float32       
 3   sssyx_low        2752 non-null   float32       
 4   sssyx_close      2752 non-null   float32       
 5   sssyx_adj_close  2752 non-null   float32       
 6   sssyx_volume     0 non-null      float32       
dtypes: datetime64[ns](1), float32(6)
memory usage: 86.7 KB


,date,sssyx_open,sssyx_high,sssyx_low,sssyx_close,sssyx_adj_close,sssyx_volume
count,2771,2752.000000,2752.000000,2752.000000,2752.000000,2752.000000,0.0
mean,2020-03-06 18:55:59.653554688,271.758911,271.758911,271.758911,271.758911,244.565323,NaN
min,2014-09-18 00:00:00,154.000000,154.000000,154.000000,154.000000,116.860001,NaN
25%,2017-06-14 12:00:00,202.100006,202.100006,202.100006,202.100006,159.970001,NaN
50%,2020-03-09 00:00:00,242.250000,242.250000,242.250000,242.250000,215.054993,NaN
75%,2022-11-28 12:00:00,331.987518,331.987518,331.987518,331.987518,313.332489,NaN
max,2025-08-27 00:00:00,487.559998,487.559998,487.559998,487.559998,487.559998,NaN
std,NaN,85.183502,85.183502,85.183502,85.183502,98.743935,NaN


## Preprocessing

### Feature selection
Since we only need one column, we'll just keep the first one.

In [28]:
oiejx = oiejx.iloc[:, :2]
sssyx = sssyx.iloc[:, :2]
prdgx = prdgx.iloc[:, :2]
rgagx = rgagx.iloc[:, :2]
vimax = vimax.iloc[:, :2]
vsiax = vsiax.iloc[:, :2]

In [29]:
sssyx

,date,sssyx_open
0,2014-09-18,170.899994
1,2014-09-19,170.800003
2,2014-09-22,169.399994
3,2014-09-23,168.500000
4,2014-09-24,169.800003
...,...,...
2766,2025-08-21,479.130005
2767,2025-08-22,486.440002
2768,2025-08-25,484.369995
2769,2025-08-26,486.399994


### Cleaning
Inspecting the missing values, we can see that those really happen at the end of the year, where dividends are usually distributed.

In [30]:
sssyx[sssyx.iloc[:, 1].isna()]['date']

68     2014-12-23
320    2015-12-23
575    2016-12-23
826    2017-12-22
828    2017-12-22
1081   2018-12-27
1083   2018-12-27
1336   2019-12-27
1337   2019-12-27
1414   2020-04-20
1591   2020-12-29
1593   2020-12-29
1844   2021-12-28
1845   2021-12-28
2098   2022-12-27
2099   2022-12-27
2349   2023-12-26
2351   2023-12-26
2604   2024-12-26
Name: date, dtype: datetime64[ns]

In [31]:
sssyx.iloc[1590:1595, :]

,date,sssyx_open
1590,2020-12-28,290.700012
1591,2020-12-29,NaN
1592,2020-12-29,284.700012
1593,2020-12-29,NaN
1594,2020-12-30,285.109985


As we can see, there aren't really missing days. It's actually that there are multiple entries for those days. So we can just drop NaN values.

In [32]:
oiejx.dropna(inplace=True)
sssyx.dropna(inplace=True)
prdgx.dropna(inplace=True)
rgagx.dropna(inplace=True)
vimax.dropna(inplace=True)
vsiax.dropna(inplace=True)
sssyx

,date,sssyx_open
0,2014-09-18,170.899994
1,2014-09-19,170.800003
2,2014-09-22,169.399994
3,2014-09-23,168.500000
4,2014-09-24,169.800003
...,...,...
2766,2025-08-21,479.130005
2767,2025-08-22,486.440002
2768,2025-08-25,484.369995
2769,2025-08-26,486.399994


### Transformation of variables
Next we need to calculate the fractional returns over every period. Since we're dealing with mutual funds, for a given asset $j$:

$R_{t,j}$ is computed as $\frac{Open_{t,j} - Open_{t-1,j}}{Open_{t-1,j}}$, that is the percent change of asset $j$ from period $t-1$ to $t$.

In [33]:
oiejx['oiejx_return'] = oiejx['oiejx_open'].pct_change()
oiejx

,date,oiejx_open,oiejx_return
0,2014-09-18,14.070000,NaN
1,2014-09-19,14.080000,0.000711
2,2014-09-22,13.970000,-0.007812
3,2014-09-23,13.880000,-0.006442
4,2014-09-24,13.970000,0.006484
...,...,...,...
2885,2025-08-21,25.850000,-0.001930
2886,2025-08-22,26.190001,0.013153
2887,2025-08-25,26.030001,-0.006109
2888,2025-08-26,26.110001,0.003073


In [34]:
sssyx['sssyx_return'] = sssyx['sssyx_open'].pct_change()
prdgx['prdgx_return'] = prdgx['prdgx_open'].pct_change()
rgagx['rgagx_return'] = rgagx['rgagx_open'].pct_change()
vimax['vimax_return'] = vimax['vimax_open'].pct_change()
vsiax['vsiax_return'] = vsiax['vsiax_open'].pct_change()

Drop the first value, as there is no percent change for that case.

In [35]:
oiejx.dropna(inplace=True)
sssyx.dropna(inplace=True)
prdgx.dropna(inplace=True)
rgagx.dropna(inplace=True)
vimax.dropna(inplace=True)
vsiax.dropna(inplace=True)
sssyx

,date,sssyx_open,sssyx_return
1,2014-09-19,170.800003,-0.000585
2,2014-09-22,169.399994,-0.008197
3,2014-09-23,168.500000,-0.005313
4,2014-09-24,169.800003,0.007715
5,2014-09-25,167.100006,-0.015901
...,...,...,...
2766,2025-08-21,479.130005,-0.003888
2767,2025-08-22,486.440002,0.015257
2768,2025-08-25,484.369995,-0.004255
2769,2025-08-26,486.399994,0.004191


Finally, we just need to confirm how many days of available data are there every year.

In [36]:
sssyx['date'].dt.year.value_counts()

date
2020    253
2015    252
2016    252
2019    252
2021    252
2024    252
2017    251
2018    251
2022    251
2023    250
2025    163
2014     72
Name: count, dtype: int64

### Merging the datasets
Extracting the returns for all assets and putting them into the matrix R

In [37]:
# Merge all dataframes on 'date' and keep only the return columns for each
returns_df = oiejx[['date', 'oiejx_return']] \
    .merge(sssyx[['date', 'sssyx_return']], on='date', how='inner') \
    .merge(prdgx[['date', 'prdgx_return']], on='date', how='inner') \
    .merge(rgagx[['date', 'rgagx_return']], on='date', how='inner') \
    .merge(vimax[['date', 'vimax_return']], on='date', how='inner') \
    .merge(vsiax[['date', 'vsiax_return']], on='date', how='inner')

returns_df.reset_index(drop=True, inplace=True)
returns_df

,date,oiejx_return,sssyx_return,prdgx_return,rgagx_return,vimax_return,vsiax_return
0,2014-09-19,0.000711,-0.000585,-0.001392,-0.000214,-0.002881,-0.007562
1,2014-09-22,-0.007812,-0.008197,-0.006689,-0.011536,-0.010750,-0.013671
2,2014-09-23,-0.006442,-0.005313,-0.005892,-0.005403,-0.006927,-0.009316
3,2014-09-24,0.006484,0.007715,0.007338,0.010430,0.007318,0.004587
4,2014-09-25,-0.014316,-0.015901,-0.014010,-0.015484,-0.013918,-0.013699
...,...,...,...,...,...,...,...
2746,2025-08-21,-0.001930,-0.003888,-0.003582,-0.003448,-0.003664,-0.001834
2747,2025-08-22,0.013153,0.015257,0.009466,0.019446,0.018698,0.031921
2748,2025-08-25,-0.006109,-0.004255,-0.007596,-0.004213,-0.007470,-0.006899
2749,2025-08-26,0.003073,0.004191,0.004066,0.006934,0.002210,0.003249


In [38]:
returns_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2751 entries, 0 to 2750
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          2751 non-null   datetime64[ns]
 1   oiejx_return  2751 non-null   float32       
 2   sssyx_return  2751 non-null   float32       
 3   prdgx_return  2751 non-null   float32       
 4   rgagx_return  2751 non-null   float32       
 5   vimax_return  2751 non-null   float32       
 6   vsiax_return  2751 non-null   float32       
dtypes: datetime64[ns](1), float32(6)
memory usage: 86.1 KB


In [39]:
R = returns_df.iloc[:,1:].values
R.shape

(2751, 6)

## Optimization

In [40]:
# Mock return matrix: 5 assets, 10 time periods
np.random.seed(0)
# T, n = 10, 5
# R = np.random.randn(T, n) * 0.01  # Simulated daily returns ~1%
T, n = R.shape

# Compute average return per asset
mu = np.mean(R, axis=0)  # Shape: (n,)

# Set a mock target return
rho = 0.1 / 250  # 10% annual return

# Define optimization variable
w = cp.Variable(n)

# Define constraints
constraints = [
    cp.sum(w) == 1,       # weights sum to 1
    mu @ w == rho         # target average return
]

# Define the objective (least squares formulation)
objective = cp.Minimize(cp.norm(R @ w - rho, 2))

# Solve the problem
problem = cp.Problem(objective, constraints)
problem.solve()

# Output results
print("Optimal weights w:", w.value)
print("Achieved average return:", mu @ w.value)
print("Portfolio risk (std dev of returns):", np.std(R @ w.value))


Optimal weights w: [-0.08221291  0.26095944  1.3650346  -0.16172342 -0.10655164 -0.27550606]
Achieved average return: 0.00039999999999999996
Portfolio risk (std dev of returns): 0.01005761879864733


In [48]:

np.sqrt(250)*mu @ w.value

np.float64(0.006324555320336758)